In [1]:
import pandas as pd
import numpy as np

import spacy

In [2]:
rwjf_grants = pd.read_json('../data/raw/rwjf_grants.json')

In [8]:
rwjf_grants.head()

,about,address,amount_awarded,awarded,awarded_on,contacts,grant_number,location,organization,page,timeframe,title,topics,website,year
0,The Robert Wood Johnson Clinical Scholars Prog...,University of North Carolina at Chapel Hill Sc...,"$865,484","$865,484",3/25/2009,Desmond Kimo Runyan,48349,"Chapel Hill, NC",University of North Carolina at Chapel Hill Sc...,1344,5/1/2009 - 4/30/2010,Technical assistance and direction for the Rob...,Health Disparities\nHealth Leadership Development,http://www.med.unc.edu/contact,2009
1,The Robert Wood Johnson Clinical Scholars Prog...,University of North Carolina at Chapel Hill Sc...,"$801,175","$801,175",3/8/2010,Desmond Kimo Runyan,48350,"Chapel Hill, NC",University of North Carolina at Chapel Hill Sc...,1201,5/1/2010 - 4/30/2011,Technical assistance and direction for the Rob...,Health Disparities\nHealth Leadership Development,http://www.med.unc.edu/contact,2010
2,This grant will continue long-standing support...,National Public Radio Inc.\n1111 North Capital...,"$2,800,000","$2,800,000",11/13/2008,Anne Gudenkauf,51491,"Washington, DC",National Public Radio Inc.,1391,11/15/2008 - 11/14/2011,Health and health care reporting by National P...,Public and Community Health\nHealth Care Quality,http://www.npr.org/,2008
3,This supplemental grant supports the nonpartis...,"George Washington University\n2121 Eye Street,...","$2,671,103","$2,671,103",7/30/2008,Judith Miller Jones,51492,"Washington, DC",George Washington University,1459,9/1/2008 - 11/30/2011,National Health Policy Forum,None,http://www.gwu.edu/,2008
4,The Foundation's Summer Medical and Dental Edu...,Association of American Medical Colleges\n655 ...,"$1,252,432","$1,252,432",8/7/2008,Charles Terrell\nRichard W. Valachovic,53039,"Washington, DC",Association of American Medical Colleges,1444,9/1/2008 - 8/31/2009,Technical assistance and direction for RWJF's ...,Social Determinants of Health\nHealth Leadersh...,http://www.aamc.org,2008


In [5]:
nlp = spacy.load('en_core_web_sm')

In [6]:
with open('../data/external/en_ranknl_long.txt', 'r') as f:
    text = f.read()
stop_words = text.split('\n')

In [43]:
'this' in stop_words

True

In [49]:
stop_words.append("\'s")

In [7]:
for stop in stop_words:
    nlp.vocab[stop].is_stop = True

In [9]:
descriptions = rwjf_grants['about']

In [28]:
docs = []

for description in descriptions:
    try:
        d = nlp(description)
    except:
        d = nlp('')
    docs.append(d)

In [32]:
t = docs[0][0]

In [50]:
tokenized_docs = []

for d in docs:
    tokens = []
    for t in d:
        if t.is_punct:
            continue
        if t.is_stop:
            continue
        if t.is_bracket:
            continue
        if t.is_quote:
            continue
        if t.like_num:
            continue
        if t.is_digit:
            continue
        if t.like_url:
            continue
        if t.like_email:
            continue
        if t.is_space:
            continue
        if t.lower_ in stop_words:
            continue
        tokens.append(t.lower_)
    tokenized_docs.append(tokens)

In [52]:
from gensim.models.phrases import Phraser, Phrases

In [53]:
from nlp_tools import GensimNGrammer

In [62]:
ngrammer = GensimNGrammer(**{'min_count': 200})

In [63]:
trigram_tokenized_docs = ngrammer.fit_transform(tokenized_docs)

In [65]:
rwjf_grants['about_tokenized'] = trigram_tokenized_docs

In [66]:
rwjf_grants.to_csv('../data/processed/rwjf_grants_05_18_2018.csv', index=False)